[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TobGerken/ISAT300/blob/main/3_HeartRateLabReview.ipynb)

# Heart Rate Lab Review 


**This notebook is published on my github. It is publicly accessible, but you cannot save your changes to my github. Learning git & github is beyond the scope of this course. If you are familiar with github, you know what to do. If you don't know github, you can save a personal copy of the file to your google drive, so that you can save your changes and can access them at a later date**

This notebook is a continuation from previous classes:

1. [GettingStarted](https://github.com/TobGerken/ISAT300/blob/main/1_GettingStarted.ipynb)
2. [Data Visualization](https://github.com/TobGerken/ISAT300/blob/main/2_DataVisualization.ipynb)


## Before we start

Some of the code today is going to be a bit complicated and I do not expect you to be able to write code like this on your own. 

However, we are covering some really important concepts and I expect you to be familiar with the underlying concepts rather than the implementation in Python. For example, we will talk about fitting models to observations from experiment and we will explore some of the pitfalls.

**I need you to know:** 

- **what goes wrong with the polynomial models and why**
- **understand the components of the exponential model**
- **why the final model appears to be a good choice**

## Motivation

You are probably used to fitting trend line to data that you collected during labs. By doing so, we create a mathematical model of our data. But why are we doing this? And How? 

Mathematical models help us generalize the behavior of experimental data, which means we can make predictions about situations that we did not observe (for example what happens between observations or after observations have ended). Fitting models to multiple sets of observations (e.g. from different test subjects) also reduces uncertainties from individual observations. 

However, there are a lot of important considerations, when creating models. It is said that 

> A bad model is worse than no model at all!

**Q: What do you think about that?**

### Learning Goals:

- Why do we need mathematical models for our observed data? 
- Use experimental data in python to apply a suitable mathematical model to the data
- Recognize the importance of finding the _right_ model
- Understand the limitations of models and the consequences of finding the wrong one. 

## Now lets get started 

The goal of today's exercise is to have a brief look at the Heart Rate data you collected during the lab. In the lab report you are be asked to perform some basic statistics on the data and to fit a model to our observations. 

In this notebook I will show you how to: 

- read in data from a google sheet into pandas
- perform basic statistics on rows and columns in a pandas data frame
- fit and evaluate different mathematical models to experimental data
- recognize the importance selecting an appropriate model 

We have previously covered how make a histogram using pandas. If you don't remember, have a look at the [Data Visualization](https://github.com/TobGerken/ISAT300/blob/main/2_DataVisualization.ipynb) lecture.

Because we are still suing pandas we have to import the pandas python module and we make it available with the name `pd`.
We will also be using [numpy](https://numpy.org/) a module that extends python for mathematics, science, and statistics. We import this as `np`.

In [ ]:
# running this will import pandas and numpy.
import pandas as pd
import numpy as np

## Reading a google spreadsheet into a dataframe

If you are using google Collab you can easily load data from google sheets, which makes sense since they are both google products. Collab provides the appropriate functions that will authenticate you with google and will then load the google sheet. Doing so will prompt a google account logon window. 

**Note that the below code will only work on Collab! If you chose to work with a different software you can also read a google sheet, but it is more complicated and it might be easier to just download the sheet to your computer as a csv or Excel file that you can then read into a dataframe.** 

In [ ]:
from google.colab import auth  # This is the module that allows google to authenticate you
import gspread                 # This is the module to interact with a google sheet.  
from google.auth import default #authenticating to google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
# This cell opens the google sheet specified in the URL, selects the first sheet, 
# reads the data, and put it in a dataframe
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1I88U6Nukk7Z86MsmYKkCRbyC2497mw0ucDasteBtxZQ/edit?usp=sharing')

worksheet =gsheets.sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# I am telling python here to use row #1 for the column names and the rest as data
df = pd.DataFrame(rows[2:], columns=rows[1])

# This bit of code is a bit complicated, but in essence I am telling python to treat all columns as numbers rather than objects
cols = list(df.columns) # this produces a list of column names 
print(cols) 

df = df[cols[:4]] # this just takes the first 4 columns (to only keep the trials )
df[cols[:4]] = df[cols[:4]].apply(pd.to_numeric) # I now apply a conversion to all columns to make sure that they are understood as numbers

print(df.shape)
df.head()

In [ ]:
# Backup code, if for some reason the google sheet is not available 
# I am creating some sample data here. 
# If you have already loaded the data from the google sheet, there is no need to do this. 
#data = {'Subject ID number':[1,2,3,4,5,6], 'Trial1':[64, 74, 66, 84, 64, 72], 
#                          'Trial2':[66, 74, 61, 74, 64, 70], 'Trial3':[64, 72, 61, 74, 69, 71]}
#df = pd.DataFrame(data)
#print(df.shape)
#df.head()

We now see that we have a dataframe with 4 columns, one for the Subject ID number and one for each trial. Now we can do some statistics on them. 

## Descriptive Statistics

The `describe()` method can always be used to calculate some of the most important descriptive statistics.  

In [ ]:
df.describe()

However, there are a lot more available in python. The table below lists the important descriptive statistics functions available for a pandas dataframe and their explanations:

    count(): The number of non-empty observations.
    sum(): The sum of the values.
    mean(): The average of the values.
    median(): The median of values.
    mode(): The mode of the values.
    std(): Standard deviation of values.
    skew(): The skewness of the values. 
    kurt(): The kurtosis (also called peakedness) of the value 
    max(): The largest of the values.
    min(): The smallest of the values.
    abs(): The absolute value of the values.
    prod(): Product of values.
    cumsum(): Cumulative sum of values.
    cumprod(): Cumulative product of values.
    

If you don't know what some of these are, I recommend to look them up on the internet. Also here is a [handy cheat-sheet with explanations of some of the most important descriptive statistics](https://res.cloudinary.com/dyd911kmh/image/upload/v1662111933/Marketing/Blog/Descriptive_Statistics_Cheat_Sheet.pdf). It does not contain things like [skewness](https://en.wikipedia.org/wiki/Skewness) though, which can also be really important for describing distributions. 

In [ ]:
# Why don't you apply one of these for example the `mean()` to the dataframe df

# Complete the code below so that it calculates the statistics for one or more columns
df

Having three trials for each resting heart rate is good, because it helps us to reduce random error in our measurements and to characterize the underlying uncertainty of the measurand. 

One way of doing so is to estimate the mean and standard deviation of the resting heart rate measurements for each subject. However, we just noticed that the `mean()` method operates on columns by default. Luckily there is a way of changing this behavior so that it operates on rows instead. 

I can calculate the average resting heart rate for each subject like this. I also assign this to a new column in the dataframe

In [ ]:
df['MeanRHR'] = df[['Trial1', 'Trial2','Trial3']].mean(axis = 1) 
# .mean(axis = 1) means to calculate the mean over rows instead of columns
df.head()

The `axis` command specifies whether the method should be performed on rows (axis = 1) or columns (axis = 0).

Now give it a try and calculate the Standard Deviation (`std()`) of the resting hear rates and save it to a new column:

In [ ]:
# Complete the code below
df['StdRHR'] = 
# Feel free to add additional columns for statistics if you would like to. 
df.head()

In [ ]:
# You can also try to create a histogram of the resting heart rates. 
# Ask yourself, which of the columns should you pick to do so. 

# Try it here. 


## Heart Rate Revocery 

We also measured during the lab how your heart rate recovers after exercise. Your recovery time depends on many factors such as your overall physical fitness, genetics, age, and so on. The rate of Heart Rate Recovery may also provide some information about your health. 

I am creating a sample dataset for Heart Rate Recovery below and read it into a new dataframe called `df_HRR`. You could also substitute in your own data (or read this in as from a spreadsheet). 

It is important to realize that you need to supply an appropriate time axis. Here I am assuming that measurements were taken every 20 seconds and to avoid me typing this our, I am using the `np.arange()` function that creates an ascending series of the same length as the heart rate samples. 

In [ ]:
import numpy as np
import pandas as pd

# I am creating a new dataframe with some realistic data here. 
# This would correspond to your measurements from the lab. 
heartrates = [187, 152 ,129 ,111 , 99 ,89 ,82,76 ,74, 71, 71, 73, 72, 71]
times = np.arange(len(heartrates))*20
print('time = %s ' % times)
df_HRR = pd.DataFrame(data={'Time':times, 'HeartRate':heartrates})
print('Shape of df: (%s, %s)'  % df_HRR.shape)
df_HRR.head(10)

In [ ]:
# Let's just plot this really quick as a plot of Time vs HearRate . 
# Remember how you can do this.
# Remember that we have to select the data frame with the Resting Heart Rate (df_HRR) 
ax1 = df_HRR.plot(kind='scatter', x='Time', y='HeartRate', grid = True)

We see that the heart rate decreases with time as we are hoping to see and that there seems to be some time period after which the decrease has leveled off to the resting heart rate. 

**Q: What do you think the resting heart rate is for this data sample?** 



### Modeling Heart Rate Recovery

Having a good model, helps us make accurate predictions of a phenomenon. Bad models can be worse than having no model at all, because they lead to decisions with disastrous outcomes. A model is only as good as the underlying data. This is often called [*garbage in garbage out*](https://en.wikipedia.org/wiki/Garbage_in,_garbage_out). 

Even more important than having good data itself, is having the expertise to select a good model. A model might fit the experimental data but may show really unexpected behavior in other circumstances. **This is bad!**

**Q: Before we continue, what kind of model would you expect to work on this data? You have already conducted the experiment have seen the data and should have made some observations.** 

In [ ]:
# You can enter your answer here. 


**Let's step back a minute to talk about models in general**

It is good practice to apply the most simple model and often that means a polynomial. 

Polynomials come in many types, also called orders, with common polynomial types being linear, quadratic or cubic:
![POLYNOMIAL ORDERS](https://editor.analyticsvidhya.com/uploads/349141.png)

The higher the order of the polynomial the more curvy it becomes and the easier it is to fit points. 

You can also fit a polynomial of order 50. This would mean that the equation to be fitted would go up to powers of 50:

$y = a_0 + a_1x^1 + a_2x^2 + ... + a_{50}x^{50}$ 

This means there are 51 coefficients to be estimates ($a_0 \ldots a_{51}$).




### Let's start fitting data. 

Numpy provides a convenient function (`np.polyfit(x,y,order)`) to fit polynomials to data. We can use this to fit a first order polynomial to the data. 

Calling `np.polyfit(x,y,1)` with our observed values of time and the desired order (1), will return a set of coefficients (i.e. the values for $a_0$ and $a_1$):

In [ ]:
# we assume that x corresponds to time and y corresponds to HeartRate 
x = df_HRR['Time']
y = df_HRR['HeartRate']
order = 1 # 1st order is a linear equation y = a*x + b
polynomialCoefficients_1 = np.polyfit(x,y,order)
print(f'The fitted equation is: y = {polynomialCoefficients_1[0]:.3f}*t + {polynomialCoefficients_1[1]:.3f}')

We can then use another function `np.polyval(<list of coefficients>, <x-values>)` to generate estimates for y at the x values from the polynomial. 

We can save these to the dataframe: 

In [ ]:
x = df_HRR['Time'] 
df_HRR['HR_Est_1']=np.polyval(polynomialCoefficients_1,x)
df_HRR.head(10)

**How about you try this for polynomials of second order (quadratic) and 25th order.**

You can always play around with other orders as well and add them to the dataframe. 

In [ ]:
# Complete the Code below to calculate the polynomial coefficients 

#Hint: This is how we fitted a first order polynom:
#polynomialCoefficients_1 = np.polyfit(x,y,1)

#complete  np.polyfit for a second order polynomial: fill in x,y, and order
x = 
y = 
order = 
polynomialCoefficients_2  = np.polyfit(x,y,order) 

# complete  np.polyfit for a 25th  order polynomial
# x and y stay the same, we just have to update the order here 
order = 
polynomialCoefficients_25 = np.polyfit(x,y,order) 

print(polynomialCoefficients_2)
print(polynomialCoefficients_25)

In [ ]:

# Now let's assign our estimated values for the heart rate to the dataframe. 

# df_HRR['HR_Est_1']=np.polyval(polynomialCoefficients_1,x)
# Complete the code below: 
df_HRR['HR_Est_2']  = np.polyval() # complete np.polyval(<coef>,x) using the 2nd order polynomial coefficients
df_HRR['HR_Est_25'] = np.polyval() # complete np.polyval(<coef>,x) using the 25th order polynomial coefficients
df_HRR.head()

Let's create a plot to see how our fits look. Beautiful!

In [ ]:
ax1 = df_HRR.plot(kind='scatter', x='Time', y='HeartRate', color = 'k')
df_HRR[['Time', 'HR_Est_1','HR_Est_2','HR_Est_25']].plot(x='Time', ax=ax1, grid = True)

The second order and 25th order polynomials seem to fit our data really well. The 25th order polynomial even connects all the measurements ... 

**Q: Is that something we would expect?**

### Evaluating our polynomials

We can put a number on our goodness of fit using the **square of the correlation coefficient (also known as $R^2$)** between the observed HeartRate and the Estimated Heart Rate for each of the models. 

Numpy has a nifty function that will give us the correlation coefficient `np.corrcoef`, which we can then square. Don't worry about the details here. You can always look up the function documentation by using `help(np.corrcoeff)` or in the [numpy manual](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html). 

In [ ]:
r2_1 = (np.corrcoef(df_HRR['HeartRate'],df_HRR['HR_Est_1'])[0,1])**2
r2_2 = (np.corrcoef(df_HRR['HeartRate'],df_HRR['HR_Est_2'])[0,1])**2
r2_25 = (np.corrcoef(df_HRR['HeartRate'],df_HRR['HR_Est_25'])[0,1])**2

print('The 1st order polynomial has an R2 of: %3.2f' % (r2_1))
print('The 2nd order polynomial has an R2 of: %3.2f' % (r2_2))
print('The 25th order polynomial has an R2 of: %3.2f' % (r2_25))


This confirms our initial impression that the higher order polynomials seem to fit the data very well. 

But there are problems. 

**Have a look at the quadratic polynomial. What do you notice?**

**Do you think that the quadratic fit will provide a good heart rate reading after let's say 2000 seconds?**

In [ ]:
# You can even use np.polyval to find out. 
HRR_Est_2000s = np.polyval(polynomialCoefficients_2,2000)
print(HRR_Est_2000s)

What might be the problem with the very high order polynomial. Well, it has a lot free parameters that can create wiggles, so it fits our data very well, but what happens if we evaluate the heart rate at a higher temporal resolution. 

Let's say instead of every 20 seconds, we estimate the heart rate every second

In [ ]:
t_high_res = np.arange(0,262) # this will create a vector of times between 0 and 262 seconds with 1 s resolution. 
y_high_res = np.polyval(polynomialCoefficients_25,t_high_res) # we now evaluate the higher order polynomial on this 

import matplotlib.pyplot as plt
ax1 = df_HRR.plot(kind='scatter', x='Time', y='HeartRate', grid = True, color = 'k')
ax1 = plt.plot(t_high_res,y_high_res) # because this is not in the dataframe I add this to the plot using pyplot. 

### Curve fitting redux. 

We now found out that random polynomials are likely not going to do the job. They may fit the observed data well, but they don't represent the underlying process of heart rate recovery and then fail. 

Here is a comic about curve fitting that illustrates my main points ... (I like [xkcd](www.xkcd.com)).
![](https://imgs.xkcd.com/comics/curve_fitting.png)

**We need to make sure that the model we fit makes physical sense. This means we need to have an expectation on how the data behaves**. 

In the case of heart rate recovery, it makes sense to think about an exponential model, where after stopping the exercise,a high heart rate rapidly declines. As the heart rate declines the rate of change will become slower. 

Exponential decay can be mathematically described like this.

$Y = a + b e^{-kt}$ ,

where a, b, and k are constants and t is time. 

This means we have 3 constants that can be changed around for us to find the best fit to our heart rate recovery data.         

#### Manual Fit

I would like you to try to fit the curve manually. This means, play around a bit by selecting values for $a$, $b$, and $k$, until you find a reasonable fit. 

I have created some code below that will help you do this. The first part defines a function in python. 

```
def ExpFunction(t, a, b, k):
    y = a + b * np.exp(-k*t) 
    return y
```

You should remember from your programming class, what a function. In short this function takes for arguments, calculates the value of a function, and returns the result. You should notice that this is the same function as the one above. 

So depending on your choice of variables $a$, $b$, and $k$, you can generate estimates for the heart rate recovery. 

I also supplied some code for a figure, that shows you the quality of the fit including the $R^2$. $R^2$ is the squared correlation coefficient.

An $R^2 = 1.00$ indicates a perfect correlation between two variables. 

While you play around, try to notice how changing each variable changes the behavior of the curve. 

**Q: What do you notice?**

In [ ]:
# This defines a function in python that returns y for provided values 
# of t, a, b, k
def ExpFunction(t, a, b, k):
    y = a + b * np.exp(-k*t) 
    return y

import matplotlib.pyplot as plt 

a = 0        # suggested starting value: 0
b = 100       # suggested starting value: 100
k = 0.1      # 0.1

t = df_HRR['Time']

y_est = ExpFunction(t, a, b, k) # Here we apply the defined function from above

ax1 = df_HRR.plot(kind='scatter', x='Time', y='HeartRate', grid = True)
ax1 = plt.plot(t,y_est, color = 'r')
ax1 = plt.text(100, 160, r"$Y = %3.2f +  %3.2f e^{- %3.2f t}$" % (a,b,k), fontsize = 16)
ax1 = plt.text(100, 140, r"$R^2$ = %3.2f" % (np.corrcoef(df_HRR['HeartRate'],y_est)[0,1])**2 , fontsize = 16)


OK, doing this by hand is tedious (possible, but tedious). You may have reached an $R^2$ of 0.99 or similar. 

Luckily, the [Scipy](https://scipy.org/) package in python contains a number of algorithms for scientific computing including curve fitting ([Example Here](https://www.geeksforgeeks.org/scipy-curve-fitting/)). 

If we apply `curve_fit()`  to our exponential function `ExpFunction` then it returns a set of fitted parameters.  

In [ ]:
from scipy.optimize import curve_fit

x = df_HRR['Time']
y = df_HRR['HeartRate']
# curve fit returns a list of fitted parameters for the supplied function
fittedParameter, variance = curve_fit(ExpFunction, x, y)
print('a= {:.2f}, b = {:.2f},  k = {:.3f}'.format(fittedParameter[0],fittedParameter[1],fittedParameter[2]))

We can then use the exponential function go generate estimates for y and plot these. As we can see, the get a great fit to our data. It is almost exactly 1.  


In [ ]:
y_est = ExpFunction(x,fittedParameter[0],fittedParameter[1],fittedParameter[2])

import matplotlib.pyplot as plt
ax1 = df_HRR.plot(kind='scatter', x='Time', y='HeartRate', grid = True)
ax1 = plt.plot(x,y_est, color = 'r')
ax1 = plt.text(100, 160, r"$Y = %3.2f +  %3.2f e^{- %3.2f }$ t" % (fittedParameter[0],fittedParameter[1],fittedParameter[2]), fontsize = 16)
ax1 = plt.text(100, 140, r"$R^2$ = %3.2f" % (np.corrcoef(y,y_est)[0,1])**2 , fontsize = 16)

## Conclusion

Curve fitting can be powerful, **when done right!** 

If done incorrectly, results can be **very misleading**. You should never chose a model, just because it fits well to your observed data. You always need to make sure that the model makes some physical sense and fits your hypothesis about the underlying data. **Know why you are doing the things you are doing!**

In Lab 3, you will again see an exponential decay model. Understanding the meaning of the parameters a, b, and k is going to be important. For example, what does $a$ indicate. What is $b$ (Hint: look at what a + b is equal to?). And what does $k$ do? 